# Procesamiento de datos de Sentinel-2

## Librerías

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob, os


## Obtener los datos

In [44]:
filesNacionalesMean = glob.glob("data/sentinel/nacional/*_mean.csv")
filesNacionalesSum = glob.glob("data/sentinel/nacional/*_sum.csv")
filesNacionalesMean

['data/sentinel/nacional\\ZonalStats_COcollection_mean.csv',
 'data/sentinel/nacional\\ZonalStats_HCHOCollection_mean.csv',
 'data/sentinel/nacional\\ZonalStats_NO2Collection_mean.csv',
 'data/sentinel/nacional\\ZonalStats_O3Collection_mean.csv',
 'data/sentinel/nacional\\ZonalStats_SO2collection_mean.csv']

### Diccionarios complementarios

In [40]:
replace_emision = {
  'pm10': '$PM_{10}$',
  'pm2.5': '$PM_{2.5}$',
  'co': '$CO$',
  'nox':'$NO_x$',
  'cov':'$COV$',
  'nh3':'$NH_3$',
  'so2':"$SO_2$",
  'no2':"$NO_2$",
  'o3':"$O_3$",
  'hcho':"$HCHO$",
}


## Preparar función de lectura

In [41]:
def readFileSentinel(path, dfFull, kind):
  cols_to_keep = ['ADM1_NAME',kind]
  name_Quimico = path.split('_')[1].lower()
  
  # remove Colletion or collection from quimico
  name_Quimico = name_Quimico.replace('collection','')
  
  df = pd.read_csv(path)
  df = df[cols_to_keep]
  df.rename(columns={kind: replace_emision[name_Quimico]}, inplace=True)

  # check if dfFull is empty
  if dfFull.empty:
    return df
  
  # merge dataframes by ADM1_NAME
  dfFull = pd.merge(dfFull, df, on='ADM1_NAME', how='inner')
  return dfFull

## Leer datos

### Datos de medias

In [42]:
dfFullMean = pd.DataFrame()

for file in filesNacionalesMean:

  dfFullMean = readFileSentinel(file, dfFullMean, 'mean')

dfFullMean

,ADM1_NAME,$CO$,$HCHO$,$NO_2$,$O_3$,$SO_2$
0,Aguascalientes,0.023545,0.000108,0.000059,0.119703,0.000064
1,Baja California,0.029563,0.000086,0.000063,0.129873,0.000201
2,Baja California Sur,0.029234,0.000085,0.000056,0.125052,0.000065
3,Campeche,0.029198,0.000113,0.000050,0.120520,0.000016
4,Chiapas,0.028332,0.000131,0.000051,0.119188,0.000042
5,Chihuahua,0.024831,0.000089,0.000057,0.125513,0.000091
6,Coahuila,0.026970,0.000098,0.000060,0.124439,0.000096
7,Colima,0.030149,0.000169,0.000057,0.119954,0.000081
8,Distrito Federal,0.026897,0.000197,0.000156,0.118464,0.000367
9,Durango,0.023602,0.000098,0.000054,0.121688,0.000059


### Datos de acumulado

In [45]:
dfFullSum = pd.DataFrame()

for file in filesNacionalesSum:

  dfFullSum = readFileSentinel(file, dfFullSum, 'sum')

dfFullSum

KeyError: "['sum'] not in index"

In [46]:
file

'data/sentinel/nacional\\ZonalStats_COcollection_sum.csv'

In [47]:
pd.read_csv(file)

,mean,Shape_Leng,ADM1_CODE,STATUS,ADM0_NAME,ADM0_CODE,EXP1_YEAR,Shape_Area,system:index,ADM1_NAME,STR1_YEAR,DISP_AREA
0,30.013227,2.805149,2028,Member State,Mexico,162,3000,0.455086,000100000000000000ca,Aguascalientes,1000,NO
1,41.699629,31.487146,2029,Member State,Mexico,162,3000,6.921294,000100000000000000cb,Baja California,1000,NO
2,38.278425,56.050239,2030,Member State,Mexico,162,3000,6.405150,000100000000000000cc,Baja California Sur,1000,NO
3,36.763867,25.072991,2031,Member State,Mexico,162,3000,4.888117,000100000000000000cd,Campeche,1000,NO
4,35.492529,15.500275,2032,Member State,Mexico,162,3000,6.231878,000100000000000000ce,Chiapas,1000,NO
5,32.622588,24.333539,2033,Member State,Mexico,162,3000,22.177385,000100000000000000cf,Chihuahua,1000,NO
6,34.812158,21.781595,2034,Member State,Mexico,162,3000,14.085113,000100000000000000d0,Coahuila,1000,NO
7,36.411485,4.021639,2035,Member State,Mexico,162,3000,0.479316,000100000000000000d1,Colima,1000,NO
8,32.876144,1.624081,2036,Member State,Mexico,162,3000,0.137609,000100000000000000d2,Distrito Federal,1000,NO
9,29.665447,19.047527,2037,Member State,Mexico,162,3000,11.093136,000100000000000000d3,Durango,1000,NO
